In [1]:
import csv

In [2]:
# Reads and Sanitizes inputs taken from the User

def sanitize_input(val):
    if(val.strip() == ""):
        return None
    if(val.lower() == "x"):
        return "x"
    elif(val.lower()=="y" or val.lower()=="yes"):
        return True
    elif(val.lower()=="n" or val.lower()=="no"):
        return False
    elif(val.lower()=="m" or val.lower()=="f"):
        return val.lower()
    else:
        return val

In [3]:
# Structure to store all data per person

class TreeNode:
    '''
    Class for holding data of each node. Contains all information related to the person in the node.
    '''        
        
    def __init__(self,name,Tree):
        '''
        Create variables for the node
        '''
        self.name = name
        self.sex = None
        self.mother = None
        self.father = None
        self.children_ref = []  
        self.tree = Tree
        self.pos = len(Tree)
        self.married = None
        
    def display(self):
        '''
        Display all the variables of the node
        '''
        
        print("\n")
        print("self.name is :{}".format(self.name))
        print("self.pos value is: {}".format(self.pos))
        print("self.sex value is: {}".format(self.sex))
        
        print("self.mother value is: {}".format(self.mother))
        print("self.father value is: {}".format(self.father))
        
        print("self.married value is: {}".format(self.married))
        print("self.children_ref value is: {}".format(self.children_ref)) 
        
        print("\n")

    def update(self , is_married = False):
        '''
        Check and update all missing variable values of the node
        '''
        
        #Check and add sex
        if(self.sex == None):
            self.sex = sanitize_input(input("Enter {}'s gender (m/f): ".format(self.name)))
         
        #Check and add Partner -> Also adds new node
        if(self.married == None and (not is_married)):
            is_married = sanitize_input(input("Is {} married? (y/n): ".format(self.name)))
        
        if(is_married):
            self.married = sanitize_input(input("{}'s spouses name: (x if not sure / empty if not needed)".format(self.name)))
        
        if(self.married!= None and is_married):
            self.tree.append(TreeNode(self.married,self.tree))
            self.married = len(self.tree) - 1
            self.tree[-1].married = self.pos
            self.tree[-1].children_ref.extend(self.children_ref)
            
            if(self.sex == "m"):
                self.tree[-1].sex = "f"
                for i in self.children_ref:
                    self.tree[i].mother = self.married
            else:
                self.tree[-1].sex = "m"
                for i in self.children_ref:
                    self.tree[i].father = self.married
        
        #Check and add Children -> Also adds new node
        if(is_married):            
            while(sanitize_input(input("Does {} have a {}th child? (y/n): ".format(self.name,len(self.children_ref)+1)))):
                print("Known children: {}".format([self.tree[i].name for i in self.children_ref]))
                child = sanitize_input(input("Enter {}'s {}'th child's name:(x if not sure / empty if not needed) ".format(self.name,len(self.children_ref)+1)))
                self.tree.append(TreeNode(child,self.tree))
                self.children_ref.append(len(self.tree) - 1)
                
                if(self.married!=None):
                    self.tree[self.married].children_ref.append(len(self.tree) - 1)

                if(self.sex == "m"):
                    self.tree[-1].father = self.pos
                    self.tree[-1].mother = self.married
                else:
                    self.tree[-1].mother = self.pos
                    self.tree[-1].father = self.married

                self.tree[-1].update()
        
        #Update family relations of spouse
        if(self.married!= None and is_married):
            self.tree[self.married].update()
            
        #Check and add mother -> Also adds new node
        flag = 0
        if(self.mother == None):    
            self.mother = sanitize_input(input("Enter {}'s mother's name: (x if not sure / empty if not needed)".format(self.name)))
            flag = 1
        
        if(flag and (self.mother != None)):
            self.tree.append(TreeNode(self.mother,self.tree))
            self.mother = len(self.tree) - 1
            self.tree[-1].children_ref.append(self.pos)
            self.tree[-1].sex = "f"
            self.tree[-1].update(is_married = True)
            
        #Check and add father -> Also adds new node 
        flag = 0
        if(self.father == None):
            self.father = sanitize_input(input("Enter {}'s father's name: (x if not sure / empty if not needed)".format(self.name)))
            flag = 1
            
        if(flag and (self.father != None)):
            self.tree.append(TreeNode(self.father,self.tree))
            self.father = len(self.tree) - 1
            self.tree[-1].children_ref.append(self.pos)
            self.tree[-1].sex = "m"
            self.tree[-1].married = self.mother
            self.tree[-1].update(is_married = True)


In [4]:
def NewTree(name):
    '''
    Creates and Initialises a new family tree 
    '''
    #List to hold all the members
    family = []
    
    #Adds first member
    family.append(TreeNode(name,family))
    
    #Iteratively adds new members
    family[0].update()
    
    return family

def SaveTree(Tree,file_name):
    '''
    Saves the tree as an csv file for record
    '''
    to_write = [["Sl_No","Name","Sex","Mother","Father","Children","Spouse"]]
    for node in Tree:
        to_write+=[[
            str(node.pos),
            str(node.name),
            str(node.sex),
            str(node.mother),
            str(node.father),
            str(node.children_ref),
            str(node.married)
        ]]
    
    with open(file_name, "w", newline='') as csvout:
        csvout = csv.writer(csvout)
        csvout.writerows(to_write)
            

In [5]:
My_family = NewTree("Adithya Jayan")

Enter Adithya Jayan's gender (m/f): m
Is Adithya Jayan married? (y/n): n
Enter Adithya Jayan's mother's name: (x if not sure / empty if not needed)Mini
Mini's spouses name: (x if not sure / empty if not needed)Jayan
Does Mini have a 2th child? (y/n): y
Known children: ['Adithya Jayan']
Enter Mini's 2'th child's name:(x if not sure / empty if not needed) Anirudh
Enter Anirudh's gender (m/f): m
Is Anirudh married? (y/n): n
Does Mini have a 3th child? (y/n): n
Enter Jayan's mother's name: (x if not sure / empty if not needed)
Enter Jayan's father's name: (x if not sure / empty if not needed)
Enter Mini's mother's name: (x if not sure / empty if not needed)
Enter Mini's father's name: (x if not sure / empty if not needed)


In [6]:
#Saves tree into file
filename = "Myfamily.csv"
SaveTree(My_family,filename)